In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
import seaborn as sns

# Task 1: Image Compression
# Task 1.1 - Lossy Compression (JPEG)
def compress_image_jpeg(image_path, quality=50):
    image = cv2.imread(image_path)
    cv2.imwrite("compressed.jpg", image, [cv2.IMWRITE_JPEG_QUALITY, quality])
    compressed = cv2.imread("compressed.jpg")
    return compressed

# Task 1.2 - Lossless Compression (PNG)
def compress_image_png(image_path):
    image = Image.open(image_path)
    image.save("compressed.png", format='PNG')
    return Image.open("compressed.png")

# Perform image compression
image_path = "/content/Dilation & Erosion.jpg"
compressed_jpeg = compress_image_jpeg(image_path)
compressed_png = compress_image_png(image_path)

In [ ]:
# Task 2: CNN on MNIST and CIFAR-10

def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Train CNN model on MNIST
mnist_model = create_cnn_model((28, 28, 1), 10)
mnist_model.fit(x_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate MNIST Model
y_pred = mnist_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Train CNN model on CIFAR-10
cifar_model = create_cnn_model((32, 32, 3), 10)
cifar_model.fit(x_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate CIFAR-10 Model
y_pred = cifar_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# ROC & AUC Curve
fpr, tpr, _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()
